# This note read an excel file with ISSN, and Source Title cols and retrive journal information from Scimago. ISSns should not contains "-".

In [47]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
df = pd.read_excel('deduplicated.xlsx')

In [48]:
len(df[['ISSN', 'Source title']].drop_duplicates().values)

2221

In [46]:
output = []
from tqdm.notebook import tqdm
s = 'Sorry, no results were found.'
for i, row in enumerate(df[['ISSN', 'Source title']].drop_duplicates().values):
    print(i)
    iss, journal = row
    print(iss)
    if type(iss) == float:
        row = ['']*8
        row[3] = journal
        row[-1] = iss
        output.append(row)
        print(iss, 't1')
        continue
    search_base = 'https://www.scimagojr.com/journalsearch.php?q='
    url = search_base+iss
    #print(url)
    res = requests.get(url).text
    soup = BeautifulSoup(res)
    if s in soup.text:
        row = ['']*8
        row[3] = journal
        row[-1] = iss
        output.append(row)
        print('No result!')
        print(iss, 't2')
        continue
    res = soup.findAll('div',{'class':"search_results"})
    base_journal_url = 'https://www.scimagojr.com/'
    try:
        journal_relative_url = [x['href'] for x in res[0].findAll('a')][0]
    except:
        row = ['']*8
        row[3] = journal
        row[-1] = iss
        output.append(row)
        print(iss, 't3')
        continue
    journal_full_url = 'https://www.scimagojr.com/'+journal_relative_url
    journal_page = requests.get(journal_full_url).text
    soup = BeautifulSoup(journal_page)
    try:
        issns = soup.findAll('div', {'class': 'journalgrid'})[0].findAll('div')[5].findAll('p')[0].text.split(', ')
    except:
        time.sleep(60)
        journal_page = requests.get(journal_full_url).text
        soup = BeautifulSoup(journal_page)
        issns = soup.findAll('div', {'class': 'journalgrid'})[0].findAll('div')[5].findAll('p')[0].text.split(', ')
    issns = [x.strip('X').strip('x') for x in issns]
    if not iss.strip('x').strip('X') in issns:
        row = ['']*8
        row[3] = journal
        row[-1] = iss
        output.append(row)
        print(iss, 't4')
        continue
    #print(journal, issn)
    s = "Sorry, no results were found."
    tabels = soup.findAll('div', {'class': 'cellslide'})
    q = [x.text for x in tabels[1].findAll('tr')[-1].findAll('td')]
    q.append(journal)
    sjr = [x.text for x in tabels[3].findAll('tr')[-1].findAll('td')][1]
    q.append(sjr)
    country = soup.findAll('div', {'class': 'journalgrid'})[0].findAll('a')[0].text
    q.append(country)
    h = soup.find('p', {'class': 'hindexnumber'}).text
    q.append(h)
    q.append(iss)
    output.append(q)
    print(q)
    time.sleep(2)

TypeError: 'NoneType' object is not subscriptable

In [40]:
pd.DataFrame(output, columns = ['field', 'year', 'Q', 'Title', 'Hindex', 'Country', 'SJR', 'issn']).to_excel('journals measures.xlsx')